<a href="https://colab.research.google.com/github/stevearonson/VB-Video-Tracking/blob/master/Detectron2_with_DeepSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Detectron2

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

## Access My Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!ls '/gdrive/My Drive/VB/Video'

##Install DeepSort

In [ ]:
!git clone --recurse-submodules https://github.com/sayef/detectron2-deepsort-pytorch.git
!cp '/gdrive/My Drive/VB/Video/ckpt.t7' detectron2-deepsort-pytorch/deep_sort/deep/checkpoint/

## Run the Detectron2-DeepSort Demo

In [ ]:
%cd detectron2-deepsort-pytorch
!python demo_detectron2_deepsort.py '/gdrive/My Drive/VB/Video/video-clip.mp4' --ignore_display
!cp demo.avi '/gdrive/My Drive/VB/Video'

Flow of video processing:

1. Iterate over video frames
2. Detect objects in frame
3. Identify players (people) inside court playing area
4. Move player detection data into deep sort data structure
5. Update deepsort algorithm
6. Draw bbox on original frame
7. Map bbox to player location on court diagram
8. Insert court diagram into original frame
9. Write new frame to output video file